In [22]:
"""
Data Collection Pipeline
Downloads 5 years of historical data for 50-100 assets using yfinance
Project: Adaptive Portfolio Manager with Advanced Optimization & Real-Time Learning
"""

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Create directory structure (if not exists)
directories = [
    'data/raw',
    'data/processed',
    'data/interim',
    'logs',
    'models',
    'results/figures',
    'results/metrics'
]

for directory in directories:
    Path(project_root / directory).mkdir(parents=True, exist_ok=True)

print("✅ Directory structure verified")
print(f"Project root: {project_root}")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d')}")


✅ Directory structure verified
Project root: /Users/aryamansingh/Desktop/adaptive_portfolio_manager
Current date: 2025-11-23


In [23]:
"""
Define 50-100 asset universe:
- 50 large-cap stocks across sectors
- 5 broad market ETFs for diversification
"""

# 50 Large-cap stocks across sectors (matching your structure)
STOCKS = {
    # Technology (10)
    'AAPL': 'Apple Inc.',
    'MSFT': 'Microsoft Corp',
    'GOOGL': 'Alphabet Inc',
    'NVDA': 'NVIDIA Corp',
    'META': 'Meta Platforms',
    'AVGO': 'Broadcom Inc',
    'ADBE': 'Adobe Inc',
    'CRM': 'Salesforce',
    'ORCL': 'Oracle Corp',
    'CSCO': 'Cisco Systems',
    
    # Finance (10)
    'JPM': 'JPMorgan Chase',
    'BAC': 'Bank of America',
    'WFC': 'Wells Fargo',
    'GS': 'Goldman Sachs',
    'MS': 'Morgan Stanley',
    'C': 'Citigroup',
    'BLK': 'BlackRock',
    'SCHW': 'Charles Schwab',
    'AXP': 'American Express',
    'USB': 'U.S. Bancorp',
    
    # Healthcare (10)
    'UNH': 'UnitedHealth Group',
    'JNJ': 'Johnson & Johnson',
    'LLY': 'Eli Lilly',
    'ABBV': 'AbbVie Inc',
    'MRK': 'Merck & Co',
    'PFE': 'Pfizer Inc',
    'TMO': 'Thermo Fisher',
    'ABT': 'Abbott Labs',
    'DHR': 'Danaher Corp',
    'AMGN': 'Amgen Inc',
    
    # Consumer (10)
    'AMZN': 'Amazon.com',
    'TSLA': 'Tesla Inc',
    'WMT': 'Walmart',
    'HD': 'Home Depot',
    'MCD': 'McDonalds',
    'NKE': 'Nike Inc',
    'SBUX': 'Starbucks',
    'TGT': 'Target Corp',
    'LOW': 'Lowes Companies',
    'TJX': 'TJX Companies',
    
    # Energy & Industrials (10)
    'XOM': 'Exxon Mobil',
    'CVX': 'Chevron Corp',
    'COP': 'ConocoPhillips',
    'BA': 'Boeing Co',
    'CAT': 'Caterpillar',
    'RTX': 'Raytheon Tech',
    'HON': 'Honeywell',
    'UPS': 'United Parcel Service',
    'GE': 'General Electric',
    'DE': 'Deere & Co',
}

# 5 Broad Market ETFs
ETFS = {
    'SPY': 'S&P 500 ETF',
    'QQQ': 'Nasdaq 100 ETF',
    'IWM': 'Russell 2000 ETF',
    'EFA': 'EAFE International ETF',
    'AGG': 'Bond Aggregate ETF',
}

# Combine all assets
ALL_ASSETS = {**STOCKS, **ETFS}
TICKERS = list(ALL_ASSETS.keys())

print(f"Total assets: {len(TICKERS)}")
print(f"Stocks: {len(STOCKS)}, ETFs: {len(ETFS)}")
print(f"\nAsset universe: {TICKERS[:10]}... (showing first 10)")

# Save asset universe for reference
asset_df = pd.DataFrame([
    {'Ticker': ticker, 'Name': name, 'Type': 'Stock' if ticker in STOCKS else 'ETF'}
    for ticker, name in ALL_ASSETS.items()
])
asset_df.to_csv(project_root / 'data/asset_universe.csv', index=False)
print(f"\n✅ Asset universe saved to data/asset_universe.csv")


Total assets: 55
Stocks: 50, ETFs: 5

Asset universe: ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'META', 'AVGO', 'ADBE', 'CRM', 'ORCL', 'CSCO']... (showing first 10)

✅ Asset universe saved to data/asset_universe.csv


In [24]:
"""
Download 5 years of daily OHLCV data with error handling
"""

# Date range: 5 years + buffer for feature calculation
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365 + 180)  # 5.5 years for buffer

print(f"Downloading data from {start_date.date()} to {end_date.date()}")
print(f"Expected trading days: ~1,260\n")

def download_data_batch(tickers, start, end, batch_size=10):
    """Download data in batches to avoid rate limits"""
    all_data = {}
    failed = []
    
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i+batch_size]
        print(f"Downloading batch {i//batch_size + 1}/{(len(tickers)-1)//batch_size + 1}: {batch}")
        
        try:
            data = yf.download(
                batch,
                start=start,
                end=end,
                progress=False,
                group_by='ticker'
            )
            
            # Store each ticker's data
            for ticker in batch:
                try:
                    if len(batch) == 1:
                        ticker_data = data
                    else:
                        ticker_data = data[ticker]
                    
                    if not ticker_data.empty and len(ticker_data) > 100:
                        all_data[ticker] = ticker_data
                        print(f"  ✅ {ticker}: {len(ticker_data)} rows")
                    else:
                        failed.append(ticker)
                        print(f"  ❌ {ticker}: Insufficient data")
                except Exception as e:
                    failed.append(ticker)
                    print(f"  ❌ {ticker}: {str(e)}")
            
            time.sleep(1)  # Rate limit protection
            
        except Exception as e:
            print(f"  ❌ Batch failed: {str(e)}")
            failed.extend(batch)
    
    return all_data, failed

# Download data
raw_data, failed_tickers = download_data_batch(TICKERS, start_date, end_date)

print(f"\n{'='*60}")
print(f"Download complete:")
print(f"  ✅ Successful: {len(raw_data)}/{len(TICKERS)}")
print(f"  ❌ Failed: {len(failed_tickers)}")
if failed_tickers:
    print(f"  Failed tickers: {failed_tickers}")


Expected trading days: ~1,260

  ✅ AAPL: 1381 rows
  ✅ MSFT: 1381 rows
  ✅ GOOGL: 1381 rows
  ✅ NVDA: 1381 rows
  ✅ META: 1381 rows
  ✅ AVGO: 1381 rows
  ✅ ADBE: 1381 rows
  ✅ CRM: 1381 rows
  ✅ ORCL: 1381 rows
  ✅ CSCO: 1381 rows
  ✅ JPM: 1381 rows
  ✅ BAC: 1381 rows
  ✅ WFC: 1381 rows
  ✅ GS: 1381 rows
  ✅ MS: 1381 rows
  ✅ C: 1381 rows
  ✅ BLK: 1381 rows
  ✅ SCHW: 1381 rows
  ✅ AXP: 1381 rows
  ✅ USB: 1381 rows
  ✅ UNH: 1381 rows
  ✅ JNJ: 1381 rows
  ✅ LLY: 1381 rows
  ✅ ABBV: 1381 rows
  ✅ MRK: 1381 rows
  ✅ PFE: 1381 rows
  ✅ TMO: 1381 rows
  ✅ ABT: 1381 rows
  ✅ DHR: 1381 rows
  ✅ AMGN: 1381 rows
  ✅ AMZN: 1381 rows
  ✅ TSLA: 1381 rows
  ✅ WMT: 1381 rows
  ✅ HD: 1381 rows
  ✅ MCD: 1381 rows
  ✅ NKE: 1381 rows
  ✅ SBUX: 1381 rows
  ✅ TGT: 1381 rows
  ✅ LOW: 1381 rows
  ✅ TJX: 1381 rows
  ✅ XOM: 1381 rows
  ✅ CVX: 1381 rows
  ✅ COP: 1381 rows
  ✅ BA: 1381 rows
  ✅ CAT: 1381 rows
  ✅ RTX: 1381 rows
  ✅ HON: 1381 rows
  ✅ UPS: 1381 rows
  ✅ GE: 1381 rows
  ✅ DE: 1381 rows
  ✅ SPY: 13

In [25]:
"""
Validate downloaded data quality
"""

def validate_data(data_dict):
    """Check for missing values, outliers, data quality issues"""
    validation_results = []
    
    for ticker, df in data_dict.items():
        # Calculate basic statistics
        returns = df['Close'].pct_change().dropna()
        
        result = {
            'ticker': ticker,
            'total_rows': len(df),
            'date_range': f"{df.index[0].date()} to {df.index[-1].date()}",
            'missing_close': df['Close'].isnull().sum(),
            'missing_volume': df['Volume'].isnull().sum(),
            'zero_volume_days': (df['Volume'] == 0).sum(),
            'price_min': f"${df['Close'].min():.2f}",
            'price_max': f"${df['Close'].max():.2f}",
            'avg_daily_return_%': returns.mean() * 100,
            'volatility_%': returns.std() * np.sqrt(252) * 100,
            'max_single_day_gain_%': returns.max() * 100,
            'max_single_day_loss_%': returns.min() * 100,
        }
        validation_results.append(result)
    
    return pd.DataFrame(validation_results)

validation_df = validate_data(raw_data)

print("Data Validation Summary:\n")
print(validation_df.head(10))
print(f"\n{'='*60}")
print(f"Average rows per ticker: {validation_df['total_rows'].mean():.0f}")
print(f"Tickers with missing close prices: {(validation_df['missing_close'] > 0).sum()}")
print(f"Tickers with zero volume days: {(validation_df['zero_volume_days'] > 0).sum()}")
print(f"Average annualized volatility: {validation_df['volatility_%'].mean():.2f}%")

# Save validation report
validation_df.to_csv(project_root / 'logs/01_data_validation_report.csv', index=False)
print(f"\n✅ Validation report saved to logs/01_data_validation_report.csv")


Data Validation Summary:

  ticker  total_rows                date_range  missing_close  missing_volume  \
0   AAPL        1381  2020-05-28 to 2025-11-21              0               0   
1   MSFT        1381  2020-05-28 to 2025-11-21              0               0   
2  GOOGL        1381  2020-05-28 to 2025-11-21              0               0   
3   NVDA        1381  2020-05-28 to 2025-11-21              0               0   
4   META        1381  2020-05-28 to 2025-11-21              0               0   
5   AVGO        1381  2020-05-28 to 2025-11-21              0               0   
6   ADBE        1381  2020-05-28 to 2025-11-21              0               0   
7    CRM        1381  2020-05-28 to 2025-11-21              0               0   
8   ORCL        1381  2020-05-28 to 2025-11-21              0               0   
9   CSCO        1381  2020-05-28 to 2025-11-21              0               0   

   zero_volume_days price_min price_max  avg_daily_return_%  volatility_%  \
0    

In [26]:
"""
Save cleaned data to CSV files in data/raw/
"""

print("Saving individual ticker files...")
saved_count = 0

for ticker, df in raw_data.items():
    # Clean column names
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    
    # Ensure index is named
    df.index.name = 'date'
    
    # Save to CSV
    filepath = project_root / f'data/raw/{ticker}.csv'
    df.to_csv(filepath)
    saved_count += 1

print(f"✅ Saved {saved_count} ticker files to data/raw/")

# Create master price matrix (Close prices only)
print("\nCreating master price matrices...")
close_prices = pd.DataFrame({
    ticker: df['close'] for ticker, df in raw_data.items()
})
close_prices.index.name = 'date'

# Save close prices
close_prices.to_csv(project_root / 'data/raw/all_close_prices.csv')
print(f"✅ Close price matrix saved: {close_prices.shape}")

# Also create volume and adjusted close matrices
adj_close_prices = pd.DataFrame({
    ticker: df['adj close'] if 'adj close' in df.columns else df['close'] 
    for ticker, df in raw_data.items()
})
adj_close_prices.index.name = 'date'
adj_close_prices.to_csv(project_root / 'data/raw/all_adj_close_prices.csv')
print(f"✅ Adjusted close price matrix saved: {adj_close_prices.shape}")

volumes = pd.DataFrame({
    ticker: df['volume'] for ticker, df in raw_data.items()
})
volumes.index.name = 'date'
volumes.to_csv(project_root / 'data/raw/all_volumes.csv')
print(f"✅ Volume matrix saved: {volumes.shape}")


Saving individual ticker files...
✅ Saved 55 ticker files to data/raw/

Creating master price matrices...
✅ Close price matrix saved: (1381, 55)
✅ Adjusted close price matrix saved: (1381, 55)
✅ Volume matrix saved: (1381, 55)


In [28]:
"""
Generate summary statistics for the dataset
"""

summary_stats = []

for ticker in raw_data.keys():
    df = raw_data[ticker]
    returns = df['close'].pct_change().dropna()
    
    # Calculate cumulative return
    total_return = (df['close'].iloc[-1] / df['close'].iloc[0]) - 1
    
    # Calculate Sharpe ratio (assuming 2% risk-free rate)
    excess_returns = returns - (0.02 / 252)
    sharpe = excess_returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
    
    # Calculate max drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    stats = {
        'Ticker': ticker,
        'Name': ALL_ASSETS.get(ticker, 'Unknown'),
        'Type': 'Stock' if ticker in STOCKS else 'ETF',
        'Data Points': len(df),
        'Start Date': df.index[0].strftime('%Y-%m-%d'),
        'End Date': df.index[-1].strftime('%Y-%m-%d'),
        'Start Price': f"${df['close'].iloc[0]:.2f}",
        'End Price': f"${df['close'].iloc[-1]:.2f}",
        'Total Return': f"{total_return * 100:.1f}%",
        'Annualized Return': f"{(((1 + total_return) ** (252 / len(df))) - 1) * 100:.1f}%",
        'Annualized Vol': f"{returns.std() * np.sqrt(252) * 100:.1f}%",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Max Drawdown': f"{max_drawdown * 100:.1f}%",
    }
    summary_stats.append(stats)

summary_df = pd.DataFrame(summary_stats)
print("="*120)
print("ASSET SUMMARY STATISTICS")
print("="*120)
print(summary_df.to_string(index=False))

# Save summary
summary_df.to_csv(project_root / 'logs/01_asset_summary_statistics.csv', index=False)
print(f"\n✅ Summary statistics saved to logs/01_asset_summary_statistics.csv")

# Print aggregate statistics
print(f"\n{'='*60}")
print("AGGREGATE STATISTICS:")
print(f"{'='*60}")
print(f"Total assets: {len(summary_df)}")
print(f"Average total return: {summary_df['Total Return'].str.rstrip('%').astype(float).mean():.1f}%")
print(f"Average Sharpe ratio: {summary_df['Sharpe Ratio'].astype(float).mean():.2f}")
print(f"Best performer: {summary_df.loc[summary_df['Total Return'].str.rstrip('%').astype(float).idxmax(), 'Ticker']}")
print(f"Worst performer: {summary_df.loc[summary_df['Total Return'].str.rstrip('%').astype(float).idxmin(), 'Ticker']}")


ASSET SUMMARY STATISTICS
Ticker                   Name  Type  Data Points Start Date   End Date Start Price End Price Total Return Annualized Return Annualized Vol Sharpe Ratio Max Drawdown
  AAPL             Apple Inc. Stock         1381 2020-05-28 2025-11-21      $77.18   $271.49       251.8%             25.8%          29.4%         0.86       -33.4%
  MSFT         Microsoft Corp Stock         1381 2020-05-28 2025-11-21     $173.19   $472.12       172.6%             20.1%          26.4%         0.75       -37.1%
 GOOGL           Alphabet Inc Stock         1381 2020-05-28 2025-11-21      $70.43   $299.66       325.5%             30.2%          30.9%         0.95       -44.3%
  NVDA            NVIDIA Corp Stock         1381 2020-05-28 2025-11-21       $8.45   $178.88      2015.7%             74.5%          51.5%         1.30       -66.3%
  META         Meta Platforms Stock         1381 2020-05-28 2025-11-21     $224.08   $594.25       165.2%             19.5%          43.2%         0.5

In [30]:
"""
Save collection metadata for tracking
"""

metadata = {
    'collection_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'start_date': start_date.strftime('%Y-%m-%d'),
    'end_date': end_date.strftime('%Y-%m-%d'),
    'total_tickers_requested': len(TICKERS),
    'successful_downloads': len(raw_data),
    'failed_downloads': len(failed_tickers),
    'failed_tickers': failed_tickers,
    'average_data_points': validation_df['total_rows'].mean(),
    'data_source': 'yfinance',
    'notes': '5 years historical data for portfolio optimization project'
}

import json
with open(project_root / 'data/raw/collection_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)

print("✅ Collection metadata saved")
print(f"\n{'='*60}")
print("01_DATA_COLLECTION.IPYNB COMPLETE")
print(f"{'='*60}")
print(f"✅ Downloaded {len(raw_data)} assets")
print(f"✅ Saved to data/raw/")
print(f"✅ Validation reports in logs/")



✅ Collection metadata saved

01_DATA_COLLECTION.IPYNB COMPLETE
✅ Downloaded 55 assets
✅ Saved to data/raw/
✅ Validation reports in logs/
